In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

def load_root_file(file_path, branches=None, print_branches=False):
    all_branches = {}
    with uproot.open(file_path) as file:
        tree = file["tree"]
        if branches is None:
            branches = tree.keys()
        if print_branches:
            print("Branches:", tree.keys())
        for branch in branches:
            try:
                all_branches[branch] = (tree[branch].array(library="np"))
            except uproot.KeyInFileError as e:
                print(f"KeyInFileError: {e}")
        all_branches['event'] = tree.num_entries
    return all_branches

file_path = "radiiCut_400_2.root"

eta_bin_edges = np.arange(0.0, 2.75, 0.25)
pt_bins       = [0.0, 5.0, np.inf]
percentiles   = [98, 99, 99.5]

In [2]:
branch_list = [
    "t5_radiiMetric",
    "t5_pt",
    "t5_pMatched",
    "t5_isFake",
    "t5_t3_idx0",
    "t5_t3_0_eta",
]

branches = load_root_file(file_path, branches=branch_list)

# Per-T5 flat arrays
metric_all  = np.concatenate(branches["t5_radiiMetric"])
pt_all      = np.concatenate(branches["t5_pt"])
pmatch_all  = np.concatenate(branches["t5_pMatched"])
isfake_all  = np.concatenate(branches["t5_isFake"])

# Get the first hit eta value for the T5, used for binning
abs_eta_first_list = []
for evt in range(branches["event"]):
    idx0_evt  = branches["t5_t3_idx0"][evt]
    eta0_evt  = branches["t5_t3_0_eta"][evt]
    abs_eta_first_list.append(np.abs(eta0_evt[idx0_evt]))
abs_eta_all = np.concatenate(abs_eta_first_list)

# Only consider 100% matched tracks
full_tracks_mask = (isfake_all == 0) & (pmatch_all > 0.95)

finite_mask = np.isfinite(metric_all) & np.isfinite(pt_all) & np.isfinite(abs_eta_all)
mask = full_tracks_mask & finite_mask

metric  = metric_all[mask]
pt      = pt_all[mask]
abs_eta = abs_eta_all[mask]

In [3]:
def compute_radii_cuts(metric, abs_eta, pt, eta_bins, percentiles, pt_bins):
    eta_centers = 0.5 * (eta_bins[:-1] + eta_bins[1:])
    cuts = {}

    for i in range(len(pt_bins) - 1):
        lo, hi = pt_bins[i], pt_bins[i+1]
        in_pt = (pt > lo) & (pt <= hi)

        cuts_bin = {}
        for p in percentiles:
            vals = []
            for j in range(len(eta_bins) - 1):
                in_eta = in_pt & (abs_eta >= eta_bins[j]) & (abs_eta < eta_bins[j+1])
                if np.any(in_eta):
                    vals.append(np.percentile(metric[in_eta], p))
                else:
                    vals.append(np.nan)
            cuts_bin[p] = np.array(vals)
        cuts[(lo, hi)] = cuts_bin

    return eta_centers, cuts

eta_centers, cuts = compute_radii_cuts(metric, abs_eta, pt, eta_bin_edges, percentiles, pt_bins)

In [4]:
def print_radii_cuts(cuts, percentiles, pt, abs_eta, metric, eta_bin_edges, eta_centers, plot=False):
    out = {}
    for (lo, hi), cuts_bin in cuts.items():
        sel = (pt > lo) & (pt <= hi)

        if plot:
            plt.figure(figsize=(9, 6))
            plt.hist2d(abs_eta[sel], metric[sel], bins=[eta_bin_edges, 60], norm=LogNorm())
            plt.colorbar(label="Counts")
            for p in percentiles:
                plt.plot(eta_centers, cuts_bin[p], marker="o", linestyle="-", label=f"{p}% retention")
            title_hi = f"{hi:g}" if np.isfinite(hi) else "inf"
            plt.xlabel("|eta| (from t5_t3_0_eta)")
            plt.ylabel("radiiCut Metric")
            plt.title(f"radiiCut vs |eta|, 100% Matched T5s, pT in ({lo:g}, {title_hi}) GeV")
            plt.grid(alpha=0.3)
            plt.legend()
            plt.tight_layout()
            plt.show()

        title_hi = f"{hi:g}" if np.isfinite(hi) else "inf"
        print(f"\npT in ({lo:g}, {title_hi}) GeV")
        for p in percentiles:
            arr = np.round(cuts_bin[p], 6)
            print(f"{p}% retention thresholds:", "{" + ", ".join(map(str, arr)) + "}")

print_radii_cuts(cuts, percentiles, pt, abs_eta, metric, eta_bin_edges, eta_centers, plot=False)


pT in (0, 5) GeV
98% retention thresholds: {0.031246, 0.031149, 0.033106, 0.039115, 0.287322, 0.690264, 0.92442, 0.495782, 0.1506, 0.166935}
99% retention thresholds: {0.059705, 0.056943, 0.05696, 0.063409, 0.357916, 0.839495, 1.082947, 0.688639, 0.181967, 0.207805}
99.5% retention thresholds: {0.098067, 0.095102, 0.098127, 0.105102, 0.433728, 0.954167, 1.195248, 0.850378, 0.218815, 0.261924}

pT in (5, inf) GeV
98% retention thresholds: {0.181548, 0.12443, 0.154407, 0.166565, 0.407317, 0.709205, 1.030456, 0.925623, 0.519899, 0.641409}
99% retention thresholds: {0.293806, 0.195468, 0.271817, 0.277988, 0.535027, 0.965708, 1.317978, 1.109844, 0.654211, 0.836095}
99.5% retention thresholds: {0.429187, 0.308526, 0.343639, 0.455679, 0.580188, 1.107541, 1.55684, 1.321647, 0.71896, 1.197171}
